In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.pyplot import imshow
sns.set(font="Droid Sans",font_scale = 1)
sns.set_style("white")
sns.set_color_codes("dark")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras_preprocessing.sequence import pad_sequences
import keras
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
import string


In [ ]:
import nltk
nltk.download('stopwords')

stwrds = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json",lines=True)

In [ ]:
# remove duplicate headlines
data=data.drop(data[data['headline'].duplicated()].index,axis=0)
sarc_cnt = len(data.query('is_sarcastic==1'))
non_sarc_cnt = len(data.query('is_sarcastic==0'))

# Summary of sarcastic lines
print(f'There are {sarc_cnt} sarcastic headlines and {non_sarc_cnt} non-sarcastic headlines')
# import stopwords from nltk
stwrds = set(stopwords.words('english'))
ps = PorterStemmer()
# method to clean a given headline by lowercasing the string, removing spaces, and removing stopwords
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds and word not in string.punctuation:
            cleaned_headline.append(ps.stem(word))
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line
data['cleaned'] = data['headline'].apply(clean_headlines)

There are 13552 sarcastic headlines and 14951 non-sarcastic headlines


In [ ]:
labels=data.is_sarcastic 
features=data.cleaned

In [ ]:
fet_train,fet_test, lab_train,lab_test = train_test_split(features,labels ,test_size =0.2)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(fet_train)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

train_sequences = tokenizer.texts_to_sequences(fet_train)
test_sequences = tokenizer.texts_to_sequences(fet_test)

21962


In [ ]:
maxlen=max([len(i) for i in train_sequences])

In [ ]:
maxlen

107

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=maxlen,  padding='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen,  padding='post')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential() 
model.add(Embedding(input_dim=vocab_size+1,output_dim=100,input_length=maxlen))
model.add(Bidirectional(tf.keras.layers.LSTM(128)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add (Dense(64,activation='relu'))
model.add (Dense(1,activation='sigmoid'))


#Compiling the model, loss: categorical crossentropy, it is the most popular for these kind of problems,
#optimizer: adam, a faster variant of the stochastic gradient method
#metrics: accuracy (We want to know the accuracy after each epoch.)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# early_stopping=EarlyStopping(patience=10, verbose=1)

#We do not want to store the weights of the model after the last epoch, we want the weights of the best model!
#We will store those weights in a file weights.hdf5
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

#We call the fit method with 100 epochs, 128 batch_size, and 0.2 validation split. 
#We have to specify which callbacks or 'plugins' we want to use.
# network_history = model.fit(fet_train, lab_train)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 107, 100)          2196300   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [ ]:
history=model.fit(train_padded, np.array(lab_train),validation_data = (test_padded,np.array(lab_test)) , epochs = 5 , verbose=2)

Epoch 1/5
713/713 - 249s - loss: 0.4809 - accuracy: 0.7562 - val_loss: 0.4080 - val_accuracy: 0.8099 - 249s/epoch - 349ms/step
Epoch 2/5
713/713 - 243s - loss: 0.2397 - accuracy: 0.9007 - val_loss: 0.4668 - val_accuracy: 0.7935 - 243s/epoch - 340ms/step
Epoch 3/5
713/713 - 244s - loss: 0.1260 - accuracy: 0.9542 - val_loss: 0.6369 - val_accuracy: 0.7811 - 244s/epoch - 343ms/step
Epoch 4/5
713/713 - 243s - loss: 0.0737 - accuracy: 0.9729 - val_loss: 0.7235 - val_accuracy: 0.7820 - 243s/epoch - 340ms/step
Epoch 5/5
713/713 - 247s - loss: 0.0440 - accuracy: 0.9839 - val_loss: 0.9027 - val_accuracy: 0.7755 - 247s/epoch - 346ms/step


In [ ]:
res.round()

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_test.to_numpy()

array([1, 1, 0, ..., 1, 1, 1])